In [399]:
import pandas as pd
import numpy as np
sample_submission_df = pd.read_csv("./dogus-teknoloji-zingat/sample_submission_zingat.csv")
test_set = pd.read_csv("./dogus-teknoloji-zingat/zingat_usecase_testset_null.csv")
train_set = pd.read_csv("./dogus-teknoloji-zingat/zingat_usecase_trainset.csv")

## Change Variable

In [400]:
def change_variable(data):
    
    data = data.copy()
    return data.rename(columns = {
        'odasayısı': 'odasayisi',
        'brütm²': 'brutm2',
        'netm²': 'netm2',
        'binayaşı': 'binayasi',
        'bulunduğukat': 'bulundugukat',
        'binadakikatsayısı': 'binadakikatsayisi',
        'isıtmatipi': 'isitmatipi',
        'İnterkom': 'interkom',
        'depremyönetmeliğineuygun': 'depremyonetmeligineuygun',
        'asansör': 'asansor',
        'Çocukoyunalanı': 'cocukoyunalani',
        'giyinmeodası': 'giyinmeodasi',
        'güncel_fiyat': 'guncelfiyat',
    })
    return data

In [401]:
train_set = change_variable(train_set)

## Converting The Yes-None Values to 0-1

In [402]:
def convert_values(data):
    
    
    data.loc[data.interkom == "VAR", "interkom"] , data.loc[data.interkom == "YOK", "interkom"] = 1,0
    data.loc[data.depremyonetmeligineuygun == "VAR", "depremyonetmeligineuygun"] , data.loc[data.depremyonetmeligineuygun == "YOK", "depremyonetmeligineuygun"] = 1,0
    data.loc[data.asansor == "VAR", "asansor"] , data.loc[data.asansor == "YOK", "asansor"] = 1,0
    data.loc[data.cocukoyunalani == "VAR", "cocukoyunalani"] , data.loc[data.cocukoyunalani == "YOK", "cocukoyunalani"] = 1,0
    data.loc[data.giyinmeodasi == "VAR", "giyinmeodasi"] , data.loc[data.giyinmeodasi == "YOK", "giyinmeodasi"] = 1,0
    data.loc[data.ebeveynbanyosu == "VAR", "ebeveynbanyosu"] , data.loc[data.ebeveynbanyosu == "YOK", "ebeveynbanyosu"] = 1,0
    
    return data

In [403]:
train_set = convert_values(train_set)

## Convert Price

In [404]:
def convert_price(data):
    
    data["guncelfiyat"] = data.guncelfiyat.str[:-3]
    data["guncelfiyat"] = data["guncelfiyat"].astype(str).astype(int)
    return data

In [405]:
train_set = convert_price(train_set)

In [406]:
def tarih_fonksiyonu(data):
   
    data = data.copy()
    data[['yil','ay','gun']] = data["tarih"].str.split("-",expand=True)
   
    data["yil"] = data["yil"].astype(str).astype(int)
    data["ay"] = data["ay"].astype(str).astype(int)
    data["gun"] = data["gun"].astype(str).astype(int)
    return data
   

In [407]:
train_set = tarih_fonksiyonu(train_set)

In [408]:
def belirtilmemis_fonksiyonu(data):
    
    data = data.copy()
    data['manzara'] = data['manzara'].replace("-","0")
    
    return data

In [409]:
train_set = belirtilmemis_fonksiyonu(train_set)

In [410]:
def path_split_fonksiyonu(data):
    
    data = data.copy()
    data[['il','ilce','mahalle']] = data["path"].str.split("/",expand=True)
    
    return data

In [411]:
train_set = path_split_fonksiyonu(train_set)

In [414]:
def odasayisi_fonksiyonu(data):
   
    data = data.copy()
    data['odasayisi'] = data['odasayisi'].replace("-","0+0")
    data[['odasayisi','salonsayisi']] = data['odasayisi'].str.split("+",expand=True)
    data["odasayisi"] = data["odasayisi"].astype(str).astype(int)
    data["salonsayisi"] = data["salonsayisi"].astype(str).astype(int)
    
    return data

In [415]:
train_set = odasayisi_fonksiyonu(train_set)

In [416]:
label_encoder_cols = ("il","ilce","mahalle","emlaktipi","binayasi","bulundugukat","binadakikatsayisi","isitmatipi","otopark","manzara")

In [417]:
from sklearn.preprocessing import LabelEncoder
def label_encoder_fonksiyonu(data,cols):
    
    for col_name in label_encoder_cols:
        labelEncoder = LabelEncoder()
        labelEncoder.fit(data[col_name].values)
        data[col_name] = labelEncoder.transform(data[col_name].values)
        
    return data

In [418]:
train_set = label_encoder_fonksiyonu(train_set,label_encoder_cols)

In [419]:
def banyosayisi_fonksiyonu(data):
    
    data = data.copy()
    data['banyosayisi'] = data['banyosayisi'].replace("6 ve üzeri","6")
    data['banyosayisi'] = data['banyosayisi'].replace("-","7")
    data['banyosayisi'] = data['banyosayisi'].astype(str).astype(int)
    
    return data

In [420]:
train_set = banyosayisi_fonksiyonu(train_set)

In [421]:
def object_to_int(data):
    
    data = data.copy()
    data[['interkom','depremyonetmeligineuygun','asansor','cocukoyunalani','giyinmeodasi','ebeveynbanyosu']] = data[['interkom',
                                                                                                                     'depremyonetmeligineuygun',
                                                                                                                     'asansor','cocukoyunalani',
                                                                                                                     'giyinmeodasi','ebeveynbanyosu']].astype(str).astype(int)
    
    return data

In [422]:
train_set = object_to_int(train_set)

In [423]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64573 entries, 0 to 64572
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   tarih                     64573 non-null  object 
 1   path                      64573 non-null  object 
 2   guncelfiyat               64573 non-null  int32  
 3   odasayisi                 64573 non-null  int32  
 4   brutm2                    64573 non-null  float64
 5   netm2                     64573 non-null  float64
 6   emlaktipi                 64573 non-null  int32  
 7   binayasi                  64573 non-null  int32  
 8   bulundugukat              64573 non-null  int32  
 9   binadakikatsayisi         64573 non-null  int32  
 10  isitmatipi                64573 non-null  int32  
 11  banyosayisi               64573 non-null  int32  
 12  manzara                   64573 non-null  int32  
 13  otopark                   64573 non-null  int32  
 14  interk